In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('train.txt', sep=';', header=None)
df.head()

,0,1
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [3]:
df.rename(columns={
    0: 'para',
    1: 'emo'
}, inplace=True)
df.head()

,para,emo
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [4]:
df.isna().sum()

para    0
emo     0
dtype: int64

## Manuall Encoding of Emotions

In [5]:
unique_emo = df['emo'].unique()
unique_emo

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [6]:
emo_label = {
    'sadness':1,
    'anger':2,
    'love':3,
    'surprise':4,
    'fear':5,
    'joy':6
}

In [7]:
df['emo'] = df['emo'].map(emo_label)
df.head()

,para,emo
0,i didnt feel humiliated,1
1,i can go from feeling so hopeless to so damned...,1
2,im grabbing a minute to post i feel greedy wrong,2
3,i am ever feeling nostalgic about the fireplac...,3
4,i am feeling grouchy,2


## Data Cleaning

### Converting to Lowercase

In [8]:
df['para'] = df['para'].apply(lambda x:x.lower())

### Removing Punctuations

#### 1) using string functions

In [9]:
import string

In [10]:
def remove_punc(txt):
    return txt.translate(str.maketrans('','',string.punctuation))

In [11]:
df['para'] = df['para'].apply(remove_punc)
df.head()

,para,emo
0,i didnt feel humiliated,1
1,i can go from feeling so hopeless to so damned...,1
2,im grabbing a minute to post i feel greedy wrong,2
3,i am ever feeling nostalgic about the fireplac...,3
4,i am feeling grouchy,2


#### 2) removing punctuations using Regular Exp.

In [12]:
import regex as rg

In [13]:
def remove_pun(txt):
    
    txt = rg.sub(r'[^a-zA-Z\s]','',txt)   # (string/pattern to be replace, replace with , text )
    # ^ : excluding these...
    # a-z : small letters
    # A-Z : large letters
    # \s : whitespace
    return txt
    # so this r'[^a-zA-Z\s]' includes punctuations,numbers...we have ^ as exclusion mark

This regex removes all of the following:

Punctuation: ! , . ?

Digits: 0–9

Emojis: 🙂 🔥 💔

Symbols: ₹ € ©

Accented letters: é ñ ü

Non-English scripts: हिंदी

In [14]:
df['para'] = df['para'].apply(remove_pun)
df['para']

0                                  i didnt feel humiliated
1        i can go from feeling so hopeless to so damned...
2         im grabbing a minute to post i feel greedy wrong
3        i am ever feeling nostalgic about the fireplac...
4                                     i am feeling grouchy
                               ...                        
15995    i just had a very brief time in the beanbag an...
15996    i am now turning and i feel pathetic that i am...
15997                       i feel strong and good overall
15998    i feel like this was such a rude comment and i...
15999    i know a lot but i feel so stupid because i ca...
Name: para, Length: 16000, dtype: object

In [15]:
df.head()

,para,emo
0,i didnt feel humiliated,1
1,i can go from feeling so hopeless to so damned...,1
2,im grabbing a minute to post i feel greedy wrong,2
3,i am ever feeling nostalgic about the fireplac...,3
4,i am feeling grouchy,2


### Removing Emojis

In [16]:
# With a manual function

def remove_emoji(txt):
    new=''
    for i in txt:
        if i.isascii(): # emoji's aren't in ascii range, 0-127 it inlcudes small/capital letters and 0-9 only
            new += i
    return new     

# similarly we can remove numbers with i.isdigit()    

In [17]:
df['para'] = df['para'].apply(remove_emoji)
df['para']

0                                  i didnt feel humiliated
1        i can go from feeling so hopeless to so damned...
2         im grabbing a minute to post i feel greedy wrong
3        i am ever feeling nostalgic about the fireplac...
4                                     i am feeling grouchy
                               ...                        
15995    i just had a very brief time in the beanbag an...
15996    i am now turning and i feel pathetic that i am...
15997                       i feel strong and good overall
15998    i feel like this was such a rude comment and i...
15999    i know a lot but i feel so stupid because i ca...
Name: para, Length: 16000, dtype: object

### Removing extra whitespace

In [18]:
def clean_whitespace(txt):
    return rg.sub(r'\s+', ' ', txt).strip()

# \s : includes whitespace, [ \t\n\r\f\v]
# \s+ : matches one or more than 1 whitespace charc, thoise charc are [ \t\n\r\f\v]
# .strip removes whitespaces in beginning and end of text...not what s in middle

In [19]:
df['para'] = df['para'].apply(clean_whitespace)

In [20]:
df['para']

0                                  i didnt feel humiliated
1        i can go from feeling so hopeless to so damned...
2         im grabbing a minute to post i feel greedy wrong
3        i am ever feeling nostalgic about the fireplac...
4                                     i am feeling grouchy
                               ...                        
15995    i just had a very brief time in the beanbag an...
15996    i am now turning and i feel pathetic that i am...
15997                       i feel strong and good overall
15998    i feel like this was such a rude comment and i...
15999    i know a lot but i feel so stupid because i ca...
Name: para, Length: 16000, dtype: object

### Removing HTML tags, for webscrapped data

we use reg.exp. r'<[^>]+>'

< is literal 1st tag..

[ ] is a character class, where ^ says exclude this, > is literal exclusion of >

so [^>] is Match any character that is NOT >

'+' is one or more charc ,Keep matching [^>] as long as possible

Final > : Matches the literal >

### Removing Stop Words
'is' 'the' 'are'

### we use NLTK - natural language toolkit

In [21]:
import nltk

In [22]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [23]:
# have to download stopwords and tokenization resources

nltk.download('punkt')
nltk.download('punkt_tab')
 # Punkt is a pre-trained sentence tokenizer
nltk.download('stopwords')
 # stopwords of all languages

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vatsc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\vatsc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vatsc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
stop_words = set(stopwords.words('english'))
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [25]:
# no. of stopwords of english..
len(stop_words)

198

### Tokenizing & Stopword removal

In [26]:
def remove_stop(txt):
    words = word_tokenize(txt)
    new = []
    for i in words:
      if i not in stop_words: # we have to check membership of i with list of stop_words...not equality !=
          new.append(i) # new += i will return each letter seperatly..good -> g,o,o,d
    return ' '.join(new) 

    # join() converts a list of strings into ONE string
    # separator.join(iterable_of_strings)
    # return new will just return a list, not string we need

In [27]:
df['para'] = df['para'].apply(remove_stop)
df['para']

0                                    didnt feel humiliated
1        go feeling hopeless damned hopeful around some...
2                im grabbing minute post feel greedy wrong
3        ever feeling nostalgic fireplace know still pr...
4                                          feeling grouchy
                               ...                        
15995        brief time beanbag said anna feel like beaten
15996    turning feel pathetic still waiting tables sub...
15997                             feel strong good overall
15998                       feel like rude comment im glad
15999                         know lot feel stupid portray
Name: para, Length: 16000, dtype: object

## Bag of Words
Bag of Words is a text vectorization technique that converts text documents into numerical vectors based on word frequency.

How it works

Build a vocabulary of all unique words in the corpus

For each document:

Count how many times each word appears

Order does not matter

Example

Documents:

"I love ML"

"I love AI"

Vocabulary:
[I, love, ML, AI]

Vectors:
| I | love | ML | AI |
| - | ---- | -- | -- |
| 1 | 1    | 1  | 0  |
| 1 | 1    | 0  | 1  |


tool - CountVectorizer

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

#### we have not yet done stemming so all words gets a unique col, all words gets included in vocabulary of BagofWords

In [29]:
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df['para'])
print('Vocabulary: ',vectorizer.get_feature_names_out())
print('\nMatrix: \n',X.toarray())

Vocabulary:  ['aa' 'aaaaaaand' 'aaaaand' ... 'zum' 'zumba' 'zz']

Matrix: 
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## Stemming
Similar words like 'accept', 'accepted', 'accepts'...same meaning with diff words

like in above vectorizer we got 'aa' 'aaaaaaand' 'aaaaand seperatly


In [30]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()


In [31]:
def stemming(txt):
    words = word_tokenize(txt)
    new = []
    for i in words:
        i = ps.stem(i)
        new.append(i)
    return ' '.join(new)    

In [32]:
df['para'] = df['para'].apply(stemming)
df['para']

0                                        didnt feel humili
1        go feel hopeless damn hope around someon care ...
2                     im grab minut post feel greedi wrong
3           ever feel nostalg fireplac know still properti
4                                             feel grouchi
                               ...                        
15995        brief time beanbag said anna feel like beaten
15996     turn feel pathet still wait tabl sub teach degre
15997                              feel strong good overal
15998                       feel like rude comment im glad
15999                         know lot feel stupid portray
Name: para, Length: 16000, dtype: object

#### now we do vectorization again after stemming...

In [40]:
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df['para'])
print('Vocabulary: ',vectorizer.get_feature_names_out()[0:50])
print('\nMatrix: \n',X.toarray())

Vocabulary:  ['aa' 'aaaaaaand' 'aaaaand' 'aaaand' 'aac' 'aahhh' 'aaron' 'ab' 'abandon'
 'abat' 'abbigail' 'abc' 'abdomen' 'abdomin' 'abduct' 'abelard' 'abhorr'
 'abid' 'abil' 'abit' 'abl' 'abnorm' 'aboard' 'abomin' 'abort' 'abou'
 'abound' 'abraham' 'abroad' 'abruptli' 'absenc' 'absolut' 'absolutli'
 'absorb' 'abstain' 'abstin' 'abstract' 'absurd' 'abt' 'abund'
 'abundantli' 'abus' 'abyss' 'ac' 'academ' 'academi' 'academia' 'acaus'
 'acceler' 'accent']

Matrix: 
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [41]:
X.toarray().shape

(16000, 10357)

In [43]:
vectorizer.get_feature_names_out().shape

(10357,)

#### n-gram vectorization...
in short - instead of bagging each word 1 by 1, we do bagging of joined words

eg-

insta is good

ngram_range(1,1)

['insta' , 'is' , 'good']

ngram_range(1,2)

['insta is' , 'is good']

Docstring:

ngram_range : tuple (min_n, max_n), default=(1, 1)
    The lower and upper boundary of the range of n-values for different
    word n-grams or char n-grams to be extracted. All values of n such
    such that min_n <= n <= max_n will be used. For example an
    ``ngram_range`` of ``(1, 1)`` means only unigrams, ``(1, 2)`` means
    unigrams and bigrams, and ``(2, 2)`` means only bigrams.
    Only applies if ``analyzer`` is not callable.


benifit is that we get more semantic value of words

## TF-IDF vectorizer

A TF-IDF score tells you how important a word is to a document, relative to the entire corpus.

“Is this word characteristic of this document, or is it just common everywhere?”

TF = Term Frequncy (how often word occurs in doc)

TF(t,d)= count of term t in document d / total words in document d

IDF = Inverse Doc Freq. (rarity of word)

IDF(t,d) = log base e ( total no. of doc in corpus /  no. of docs having t )

TF*IDF = tfidf score

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [49]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['para'])
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [50]:
vectorizer.get_feature_names_out()[0:50]

array(['aa', 'aaaaaaand', 'aaaaand', 'aaaand', 'aac', 'aahhh', 'aaron',
       'ab', 'abandon', 'abat', 'abbigail', 'abc', 'abdomen', 'abdomin',
       'abduct', 'abelard', 'abhorr', 'abid', 'abil', 'abit', 'abl',
       'abnorm', 'aboard', 'abomin', 'abort', 'abou', 'abound', 'abraham',
       'abroad', 'abruptli', 'absenc', 'absolut', 'absolutli', 'absorb',
       'abstain', 'abstin', 'abstract', 'absurd', 'abt', 'abund',
       'abundantli', 'abus', 'abyss', 'ac', 'academ', 'academi',
       'academia', 'acaus', 'acceler', 'accent'], dtype=object)

In [53]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer()
X_count = cv.fit_transform(df['para'])

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['para'])

print(X_count[0].toarray())
print(X_tfidf[0].toarray())


[[0 0 0 ... 0 0 0]]
[[0. 0. 0. ... 0. 0. 0.]]


In [54]:
df.head()

,para,emo
0,didnt feel humili,1
1,go feel hopeless damn hope around someon care ...,1
2,im grab minut post feel greedi wrong,2
3,ever feel nostalg fireplac know still properti,3
4,feel grouchi,2


In [57]:
X[0].nnz
# non zero in 1st doc

3

In [59]:
# displaying 1st doc where its nonzero
pd.DataFrame(
    X[0].toarray(),
    columns=vectorizer.get_feature_names_out()
).loc[:, lambda df: df.iloc[0] != 0]


,didnt,feel,humili
0,0.600113,0.118816,0.791041
